Installation

In [1]:
pip install opencv-python mediapipe numpy PyOpenGL glfw

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import mediapipe as mp
import math
import numpy as np

# Impor pustaka 3D
from OpenGL.GL import *
from OpenGL.GLUT import *
import glfw

Inisialisasi

In [2]:
# Inisialisasi MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Buka kamera
cap = cv2.VideoCapture(0)

# Variabel untuk Zoom & Rotate
initial_distance = None  
initial_angle = None
zoom_level = 1.0        
rotation_angle = 0.0

Fungsi - fungsi

In [3]:
# --- Fungsi untuk Mengatur OpenGL ---
def setup_gl(width, height):
    glClearColor(0.0, 0.0, 0.0, 0.0)
    glEnable(GL_DEPTH_TEST)

    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    glOrtho(-1.0, 1.0, -1.0, 1.0, -1.0, 1.0)

    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()

In [4]:
# Fungsi untuk memuat file OBJ
def load_obj(filename):
    vertices = []
    faces = []
    for line in open(filename, 'r'):
        if line.startswith('v '):
            vertices.append(list(map(float, line.strip().split()[1:4])))
        elif line.startswith('f '):
            face_indices = line.strip().split()[1:]
            faces.append([int(i.split('/')[0]) - 1 for i in face_indices])
    return vertices, faces

try:
    vertices, faces = load_obj('Rubik.obj')
except FileNotFoundError:
    print("File 3D tidak ditemukan! Pastikan 'your_model.obj' berada di folder yang sama.")
    exit()

In [5]:
# --- Fungsi untuk Merender Objek 3D ---
def draw_obj():
    glBegin(GL_TRIANGLES)
    glColor3f(0.8, 0.8, 0.8) # Warna objek
    for face in faces:
        for vertex_index in face:
            glVertex3fv(vertices[vertex_index])
    glEnd()

Main loop

In [6]:
# Inisialisasi GLFW
if not glfw.init():
    exit()

window = glfw.create_window(1280, 720, "Hand Gesture 3D Control", None, None)
if not window:
    glfw.terminate()
    exit()

glfw.make_context_current(window)
setup_gl(1280, 720)

while not glfw.window_should_close(window):
    # Dapatkan frame dari kamera
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Proses deteksi tangan
    result = hands.process(frame_rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # === Logika Zoom (Jari Telunjuk & Jempol) ===
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            
            h, w, c = frame.shape
            index_x, index_y = int(index_tip.x * w), int(index_tip.y * h)
            thumb_x, thumb_y = int(thumb_tip.x * w), int(thumb_tip.y * h)
            
            current_distance = math.hypot(index_x - thumb_x, index_y - thumb_y)
            
            if initial_distance is None:
                initial_distance = current_distance
            
            zoom_level = current_distance / initial_distance

            # === Logika Rotasi (Pergelangan & Jari Tengah) ===
            wrist = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST]
            middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]

            wrist_x, wrist_y = int(wrist.x * w), int(wrist.y * h)
            middle_x, middle_y = int(middle_tip.x * w), int(middle_tip.y * h)

            angle_rad = math.atan2(middle_y - wrist_y, middle_x - wrist_x)
            angle_deg = math.degrees(angle_rad) + 90
            
            if initial_angle is None:
                initial_angle = angle_deg

            rotation_angle = angle_deg - initial_angle
            
            if rotation_angle > 180:
                rotation_angle -= 360
            elif rotation_angle < -180:
                rotation_angle += 360

    # === Rendering OpenGL ===
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glLoadIdentity()
    
    # Terapkan transformasi (rotasi dan skala/zoom)
    glRotatef(rotation_angle, 0, 0, 1)
    glScalef(zoom_level, zoom_level, zoom_level)
    
    # Gambar objek 3D
    draw_obj()

    glfw.swap_buffers(window)
    glfw.poll_events()

# Hancurkan window dan akhiri
cap.release()
glfw.terminate()

C:\Users\chris\AR\venv_312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Logika